This notebook demonstrates the common usage pattern for defining structs (called Datatypes) in Z3 using the Python API. Consider using this as an alternative to defining multiple variables for _e.g._ the address, data, and enable signals of a memory's write port.

In [1]:
import z3

In [3]:
bitwidth = 4
addrwidth = 4

dt = z3.Datatype('MemWritePort')
dt.declare('memwr', ('addr', z3.BitVecSort(addrwidth)),
                    ('data', z3.BitVecSort(bitwidth)),
                    ('wr_en', z3.BoolSort()))
dt = dt.create()

`MemWritePort` is the name of the data type. `memwr` _is the **constructor** for_ `MemWritePort`

In [4]:
dt.memwr

memwr

In [10]:
v = dt.memwr(5, 2, True)
v

memwr(5, 2, True)

`addr`, `data`, `wr_en` _are **projections** used to access its members_

In [11]:
dt.addr(v)

addr(memwr(5, 2, True))

In [12]:
z3.simplify(dt.addr(v))

5

_Constructor arguments may be symbolic values, and you can make symbolic values for structs._

In [16]:
x, y = z3.BitVecs("x y", addrwidth)

In [18]:
w = dt.memwr(x + y, 9, x > y)
w

memwr(x + y, 9, x > y)

In [19]:
port = z3.Const("mem:wport:1", dt)

In [21]:
dt.addr(port), dt.data(port), dt.wr_en(port)

(addr(mem:wport:1), data(mem:wport:1), wr_en(mem:wport:1))

_**Tip.** Here is some memory boilerplate._

In [24]:
mem = z3.Array("mem", z3.BitVecSort(addrwidth), z3.BitVecSort(bitwidth))
mem_p = z3.If(dt.wr_en(port), z3.Store(mem, dt.addr(port), dt.data(port)), mem)

In [25]:
mem_p

If(wr_en(mem:wport:1),
   Store(mem, addr(mem:wport:1), data(mem:wport:1)),
   mem)

_Nice._